In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm import tqdm
import matplotlib.pyplot as plt
import shutil
from itertools import repeat

In [ ]:
def load_folder(folder, folder_name, PATH_single_folder):
    path, files = open_folder(folder, PATH_single_folder)
    distance = pd.DataFrame()
    for file in tqdm(files):
        if re.search(f'^{folder_name}-', file):
            df = pd.read_json(f"{path}/{file}")
            distance = pd.concat([df,distance], axis = 0)
#     distance = distance.sort_values(by='dateTime')
    return distance

In [ ]:
path_to_folder = 'D:/DATA_THESIS/Projekt_nw_AG_AJ/Outputs_project_data_preprocessing'


In [ ]:
PATH = "C:/Users/Admin/Studia_II/2semsestr/Praca magisterska/all_fitbits"
folder = 'Physical Activity'
path_to_save = f'{path_to_folder}/Outputs_project_data_preprocessing/2_fitbit_sleep'

In [ ]:
import os
import zipfile
from fnmatch import fnmatch
import re


def unzip_patient(iteration, path = PATH):
    fitbit_patients = os.listdir(f"{PATH}/{iteration}")
    folders_names = []
    for file in fitbit_patients:
            if os.path.exists(f"{PATH}/{iteration}/{file.strip('.zip')}")==0:
                with zipfile.ZipFile(f"{PATH}/{iteration}/{file}","r") as zip_ref:
                    zip_ref.extractall(f"{PATH}/unzipped/{iteration}/{file.strip('.zip')}")
                    folders_names.append(file.strip('.zip'))
    # remove pdf
    for dirpath, dirnames, filenames in os.walk(f"{PATH}/unzipped"):
        for file in filenames:
            if fnmatch(file, '*.pdf'):
                os.remove(os.path.join(dirpath, file))

    patient_folders = []
    for patient in folders_names:
        PATH_each_patient_data = f"{PATH}/unzipped/{iteration}/{patient}"
        patient_folders.append(os.listdir(PATH_each_patient_data)[0])
    return patient_folders , folders_names


patient_nr = 0

def go_to_patient_and_open_folder(iteration, patient_nr, folder, folders_names, patient_folders):
    patient = os.listdir(f"{PATH}/unzipped/{iteration}/{folders_names[patient_nr]}")
    for i in range(0, len(patient)):
        PATH_single_folder = f"{PATH}/unzipped/{iteration}/{folders_names[patient_nr]}/{patient[i]}"
        fitbit_patient_folders = os.listdir(PATH_single_folder)
        PATH_folder_for_measurment = f"{PATH_single_folder}/{folder}"
        choosen_folder = os.listdir(PATH_folder_for_measurment)
#         print("choosen folder: ", folder)
    #     check_data(PATH_folder_for_measurment, choosen_folder)
        clean_folders(fitbit_patient_folders, PATH_folder_for_measurment,PATH_single_folder)
        print(PATH_single_folder)
        
        heart_rate = load_folder("Physical Activity", 'heart_rate', PATH_single_folder)
        print("========== LENGTH OF HEART_RATE DATAFRAME",len(heart_rate),' ==========\n\n')
        return fitbit_patient_folders, choosen_folder,PATH_single_folder, PATH_folder_for_measurment


def check_data(PATH_folder_for_measurment, choosen_folder):
    empty_files = []
    files_to_keep = []
    for i in choosen_folder:
        PATH_for_file_in_meas_folder = f"{PATH_folder_for_measurment}/{i}"
        dataframe_for_file_in_meas_folder = pd.read_csv(PATH_for_file_in_meas_folder)
#         print("Dataframe has lenght: ", len(dataframe_for_file_in_meas_folder))
        if len(dataframe_for_file_in_meas_folder)==0:
            empty_files.append(PATH_for_file_in_meas_folder)
        else:
            files_to_keep.append(PATH_for_file_in_meas_folder)
    print('Empty files: ', len(empty_files), "Files with data: ", len(files_to_keep))
    

def clean_folders(fitbit_patient_folders, PATH_folder_for_measurment, PATH_single_folder):
    empty_files_all = []
    files_to_keep_all = []
    dict_stats = {}
    file_name_to_keep=[]
    for m_folder in fitbit_patient_folders:
#         print(f"\n-------- FOLDER {m_folder}--------")
        PATH_folder_for_measurment = f"{PATH_single_folder}/{m_folder}"
        choosen_folder = os.listdir(PATH_folder_for_measurment)
    #     print(choosen_folder)
        dict_stats[m_folder] = len(choosen_folder)
        for i in choosen_folder:
#             print(i)
            PATH_for_file_in_meas_folder = f"{PATH_folder_for_measurment}/{i}"
            if re.search('README', i) or re.search('Readme', i) or re.search('Media', i):
                dataframe_for_file_in_meas_folder = []
            else:
#                 print(PATH_for_file_in_meas_folder)
                dataframe_for_file_in_meas_folder = pd.read_csv(PATH_for_file_in_meas_folder,on_bad_lines='skip')
#                 print("\tDataframe has lenght: ", len(dataframe_for_file_in_meas_folder))

            if len(dataframe_for_file_in_meas_folder)==0:
                empty_files_all.append(PATH_for_file_in_meas_folder)

                if re.search('Media', i):
#                     print("deleted folder")
                    shutil.rmtree(PATH_for_file_in_meas_folder)
                else:
#                     print("deleted file")
                    os.remove(PATH_for_file_in_meas_folder)

            else:
                files_to_keep_all.append(PATH_for_file_in_meas_folder)
                file_name_to_keep.append(i)
    choosen_folder = os.listdir(PATH_folder_for_measurment)
#     print(choosen_folder)
    dict_stats[m_folder] = len(choosen_folder)
    plt.figure(figsize=(20,4))
    plt.bar(dict_stats.keys(), dict_stats.values())
    plt.show()
    
    
def open_folder(folder, PATH_single_folder):
    PATH_folder_for_measurment = f"{PATH_single_folder}/{folder}"
    choosen_folder = os.listdir(PATH_folder_for_measurment)
    return PATH_folder_for_measurment, choosen_folder

In [ ]:
def process_fitbit_iteration(iterations):
    list_of_patients = []
    df_with_all_sleep_data_for_iteration = pd.DataFrame()
    for iteration in iterations:
        patient_folders , folders_names = unzip_patient(iteration, PATH)
        for i in range(0,len(folders_names)):
            PATH_single_folder = f"{PATH}/unzipped/{iteration}/{folders_names[i]}/{patient_folders[i]}"
            heart_rate = load_folder("Physical Activity", 'heart_rate', PATH_single_folder)
            df_with_all_sleep_data_for_iteration = pd.concat([df_with_all_sleep_data_for_iteration, heart_rate], axis = 0)
            list_of_patients.extend(repeat(folders_names[i], len(heart_rate)))
    df_with_all_sleep_data_for_iteration = df_with_all_sleep_data_for_iteration.reset_index(drop = True)
    df_with_all_sleep_data_for_iteration['patient_id'] = list_of_patients
    display(df_with_all_sleep_data_for_iteration.head())

    # prepare columns for future process
    # Split 'value' column into separate columns
    df_with_all_sleep_data_for_iteration[['bpm', 'confidence']] = pd.DataFrame(df_with_all_sleep_data_for_iteration['value'].tolist(), index=df_with_all_sleep_data_for_iteration.index)
    
    # Drop the original 'value' column
    df_with_all_sleep_data_for_iteration = df_with_all_sleep_data_for_iteration.drop('value', axis=1)
    
    # Convert 'dateTime' column to datetime type
    df_with_all_sleep_data_for_iteration['dateTime'] = pd.to_datetime(df_with_all_sleep_data_for_iteration['dateTime'])

    # filter data with confidence equal or higher than 2 
    df_with_all_sleep_data_for_iteration_conf = df_with_all_sleep_data_for_iteration[df_with_all_sleep_data_for_iteration['confidence']>=2]
    print("The amount of samples with confidence 1 and 0: ", len(df_with_all_sleep_data_for_iteration) - len(df_with_all_sleep_data_for_iteration_conf),
         "which is ", (len(df_with_all_sleep_data_for_iteration) - len(df_with_all_sleep_data_for_iteration_conf))*100/len(df_with_all_sleep_data_for_iteration), "%")



    # repeat for sleep
    list_of_patients = []
    df_with_all_sleep = pd.DataFrame()
    for iteration in iterations:
        patient_folders , folders_names = unzip_patient(iteration, PATH)
        for i in range(0,len(folders_names)):
            PATH_single_folder = f"{PATH}/unzipped/{iteration}/{folders_names[i]}/{patient_folders[i]}"
            sleep = load_folder("Sleep", "sleep",PATH_single_folder)
            df_with_all_sleep = pd.concat([df_with_all_sleep, sleep], axis = 0)
    df_with_all_sleep = df_with_all_sleep.reset_index(drop = True)

    df_night_sleep = pd.DataFrame()
    for i in range(0, len(df_with_all_sleep)):
        df_one_night = pd.DataFrame(df_with_all_sleep['levels'][i]['data'])
        df_night_sleep = pd.concat([df_night_sleep, df_one_night,] , axis = 0)
    df_night_sleep = df_night_sleep.sort_values('dateTime').reset_index(drop = True)
    df_night_sleep
    
    df_night_sleep['dateTime'] = pd.to_datetime(df_night_sleep['dateTime'])
    
    time_diff = df_night_sleep['dateTime'].diff()
    
    # Identify the moments where the difference is more than 1 hour
    gap_mask = time_diff > pd.Timedelta('1 hour')
    
    # Create a new column 'segment' with a cumulative sum
    df_night_sleep['segment'] = gap_mask.cumsum()

    return df_with_all_sleep_data_for_iteration_conf, df_with_all_sleep, df_night_sleep

In [ ]:
df_with_all_sleep_data_for_iteration_conf,df_with_all_sleep, df_night_sleep = process_fitbit_iteration(['iteration_02'] )

In [ ]:
iterations = [
            ['iteration_01'], 
              ['iteration_02'],
              ['iteration_03'], 
               ['iteration_04'], 
              ['iteration_05'],
                ['iteration_06']
    ]

iter_numbers = [1,2,3,4,5,6]
for iter, iter_nr in zip(iterations, iter_numbers):
    df_with_all_sleep_data_for_iteration_conf,df_with_all_sleep, df_night_sleep = process_fitbit_iteration(iter)
    # Convert strings to datetime objects
    df_with_all_sleep['startTime'] = pd.to_datetime(df_with_all_sleep['startTime'])
    
    # merge each bmp with sleep event
    df_with_all_sleep['endTime'] = pd.to_datetime(df_with_all_sleep['endTime'])
    df_with_all_sleep_data_for_iteration_conf['dateTime'] = pd.to_datetime(df_with_all_sleep_data_for_iteration_conf['dateTime'])
    df_with_all_sleep_data_for_iteration_conf = df_with_all_sleep_data_for_iteration_conf.sort_values('dateTime')
    df_with_all_sleep = df_with_all_sleep.sort_values('startTime')
    merged_df = pd.merge_asof(df_with_all_sleep_data_for_iteration_conf,df_with_all_sleep, left_on='dateTime', right_on='startTime')
    
    
    # Filter the rows based on the condition
    filtered_df = merged_df[(merged_df['dateTime'] >= merged_df['startTime']) & 
                            (merged_df['dateTime'] <= merged_df['endTime'])]
    
    # Add a new column for labels
    filtered_df['label'] = filtered_df['endTime']
    
    # Drop unnecessary columns
    # filtered_df = filtered_df.drop(['startTime', 'endTime'], axis=1)
    
    # Display the result
    filtered_df

    merged_df = pd.merge_asof(filtered_df,df_night_sleep, on='dateTime')
    merged_df
    merged_df.to_pickle(f"{path_to_save}/{iter}_sleep.pkl")
    df_with_all_sleep.to_pickle(f'{path_to_save}/sleep_joined_i0{iter_nr}.pkl')
    df_night_sleep.to_pickle(f'{path_to_save}/phases_joined_i0{iter_nr}.pkl')
    df_with_all_sleep_data_for_iteration_conf.to_pickle(f'{path_to_save}/heart_rate_joined_i0{iter_nr}.pkl')

# Load

In [ ]:
df_with_all_sleep = pd.DataFrame()
for i in range(1, 6):
    df_with_sleep = pd.read_pickle(f'{path_to_save}/sleep_joined_i0{i}.pkl')
    df_with_all_sleep = pd.concat([df_with_all_sleep,df_with_sleep], axis = 0)

In [ ]:
df_with_all_sleep_data_for_iteration_conf = pd.DataFrame()
for i in range(1, 6):
    df_with_sleep_data_for_iteration_conf = pd.read_pickle(f'{path_to_save}/heart_rate_joined_i0{i}.pkl')
    df_with_all_sleep_data_for_iteration_conf = pd.concat([df_with_all_sleep_data_for_iteration_conf,df_with_sleep_data_for_iteration_conf], axis = 0)

In [ ]:
df_with_all_sleep_data_for_iteration_conf['confidence'].value_counts().keys()

In [ ]:
df_with_all_sleep_data_for_iteration_conf['confidence'].unique()

In [ ]:
# Create a bar chart
dict_with_stats = df_with_all_sleep_data_for_iteration_conf['confidence'].value_counts()
fig = px.bar(
    x=dict_with_stats.keys(),
    y=dict_with_stats.values,
    labels={'y': 'Count', 'x': 'Confidence'},
)
# Update layout
fig.update_layout(
    xaxis_title='Confidence',
    yaxis_title='Count',
    template='plotly_white',
    title = 'Statistics of the confidence column'
)

# Show the plot
fig.show()

In [ ]:
# Create a bar chart
dict_with_stats = df_with_all_sleep['logType'].value_counts()
fig = px.bar(
    x=dict_with_stats.keys(),
    y=dict_with_stats.values,
    labels={'y': 'Count', 'x': 'value'},
)
# Update layout
fig.update_layout(
    xaxis_title='Confidence',
    yaxis_title='Count',
    template='plotly_white',
    title = 'Statistics of the logType column'
)

# Show the plot
fig.show()

In [ ]:
# Create a bar chart
dict_with_stats = df_with_all_sleep['mainSleep'].value_counts()
fig = px.bar(
    x=dict_with_stats.keys(),
    y=dict_with_stats.values,
    labels={'y': 'Count', 'x': 'value'},
)
# Update layout
fig.update_layout(
    xaxis_title='Confidence',
    yaxis_title='Count',
    template='plotly_white',
    title = 'Statistics of the mainSleep column'
)

# Show the plot
fig.show()

In [ ]:
# Calculate the total length of each phase
df = df_night_sleep.copy()
df['total_length'] = df.groupby('level')['seconds'].transform('sum')

# Drop duplicate rows to keep only unique phase entries
unique_phases = df[['level', 'total_length']].drop_duplicates()
# Calculate the percentage of total length for each phase
unique_phases['percentage'] = (unique_phases['total_length'] / unique_phases['total_length'].sum()) * 100
unique_phases['percentage'] = unique_phases['percentage'].round(1)

# Create a bar chart
fig = px.bar(unique_phases, x='level', y='total_length',
             text='percentage',
             labels={'total_length': 'Total Length', 'phase': 'Phase'},
             title='Total Length of Each Phase',
             template='plotly_white')

# Show the plot
fig.show()

In [ ]:
# Calculate the total length of each phase
# Generate new columns for hours, minutes, and seconds
df['dateTime'] = pd.to_datetime(df['dateTime'])
df['hour'] = df['dateTime'].dt.hour
df['minute'] = df['dateTime'].dt.minute
df['second'] = df['dateTime'].dt.second

# Create a bar chart to visualize the distribution of values at different times
fig = px.bar(df.groupby(['hour', 'minute', 'second', 'level']).size().reset_index(name='count'),
             x='hour', y='count',
             color='level',
             labels={'count': 'Count', 'hour': 'Hour'},
             title='Distribution of Values at Different Times',
             template='plotly_white')

# Show the plot
fig.show()

In [ ]:
df_night_sleep['level'].unique()

In [ ]:
df_with_all_sleep['startTime'] = pd.to_datetime(df_with_all_sleep['startTime'])
df_with_all_sleep['endTime'] = pd.to_datetime(df_with_all_sleep['endTime'])

In [ ]:
# drop duplicates in start/end times
len_before = len(df_with_all_sleep)

print('Length before droping the repeated values: ', len(df_with_all_sleep))
# duplicate_rows = general_sleep[general_sleep.duplicated(subset='startTime', keep=False)]
# display(duplicate_rows)

df_with_all_sleep = df_with_all_sleep.drop_duplicates(subset='startTime')
print('Length after droping the repeated values: ', len(df_with_all_sleep), 'removed percentage: ',
     100-(len(df_with_all_sleep)*100/len_before),'%')


In [ ]:
patient = df_with_all_sleep_data_for_iteration_conf[df_with_all_sleep_data_for_iteration_conf['patient_id'] 
    ==df_with_all_sleep_data_for_iteration_conf['patient_id'].unique()[15]].sort_values(by='dateTime').reset_index(drop=True)


In [ ]:
import plotly.express as px
import numpy as np
general_sleep = df_with_all_sleep[df_with_all_sleep['patient_id']==df_with_all_sleep_data_for_iteration_conf['patient_id'].unique()[15]].sort_values(by='dateOfSleep').reset_index(drop=True)


fig = px.bar(
#     general_sleep,
    x=general_sleep['startTime'],
    y=np.repeat(patient['bpm'].max(), len(general_sleep)),
#     color=np.repeat('green', len(general_sleep)),
    # Uncomment and adjust other parameters as needed
)

fig.add_bar(
#     general_sleep,
    x=general_sleep['endTime'],
    y=np.repeat(patient['bpm'].max(), len(general_sleep)),
    name= 'end'
#     color=np.repeat('red', len(general_sleep)),
    # Uncomment and adjust other parameters as needed
)

fig.show()


# Check how many sleeps are less than 3h

In [ ]:
df_with_all_sleep = df_with_all_sleep.reset_index(drop =True)
df_long_sleep = df_with_all_sleep.copy()
for i in range(0, len(df_with_all_sleep)):
    if( df_with_all_sleep['endTime'][i] - df_with_all_sleep['startTime'][i] )<= pd.Timedelta('3 hour'):
        df_long_sleep = df_long_sleep[df_long_sleep['startTime']!= df_with_all_sleep['startTime'][i]].reset_index(drop =True)
        print('Start time: ', df_with_all_sleep['startTime'][i], 
              'Duration: ', ( df_with_all_sleep['endTime'][i] - df_with_all_sleep['startTime'][i] ))

In [ ]:
print('All sleep data: ',len(df_with_all_sleep),
     "Only long sleep data: ", len(df_long_sleep),
     'Percentage of removed data: ', 100-(100*len(df_long_sleep))/len(df_with_all_sleep), '%' )

# Check nights and generate the dataframe with only sleep bmp

In [ ]:
df_long_sleep

In [ ]:
df = df_with_all_sleep.copy()
# [df_with_all_sleep['type'] == 'classic']

# Calculate the total length of each phase
# Generate new columns for hours, minutes, and seconds
df['dateTime'] = pd.to_datetime(df['startTime'])
df['hour'] = df['startTime'].dt.hour
df['minute'] = df['startTime'].dt.minute
df['second'] = df['startTime'].dt.second

# Create a bar chart to visualize the distribution of values at different times
fig = px.bar(df.groupby(['hour', 'minute', 'second', 'type']).size().reset_index(name='count'),
             x='hour', y='count',
             color='type',
             labels={'count': 'Count', 'hour': 'Hour'},
             title='Distribution of Values at Different Times',
             template='plotly_white')

# Show the plot
fig.show()

In [ ]:
import pandas as pd

new_df_filtered = pd.DataFrame()

df_long_sleep['startTime'] = pd.to_datetime(df_long_sleep['startTime'])
df_long_sleep['endTime'] = pd.to_datetime(df_long_sleep['endTime'])
df_with_all_sleep_data_for_iteration_conf['dateTime'] = pd.to_datetime(df_with_all_sleep_data_for_iteration_conf['dateTime'])
number = 0
for p in tqdm(df_with_all_sleep_data_for_iteration_conf['patient_id'].unique()):
    
    df_bpm = df_with_all_sleep_data_for_iteration_conf[
            df_with_all_sleep_data_for_iteration_conf['patient_id'] == p].sort_values(by='dateTime').reset_index(drop=True)
    df_sleep = df_long_sleep[df_long_sleep['patient_id'] == p].sort_values(by='startTime').reset_index(drop=True)
    
    for sleep_nr in range(0, len(df_sleep)):
        condition = (
            (df_bpm['dateTime'] >= df_sleep['startTime'][sleep_nr]) &
            (df_bpm['dateTime'] <= df_sleep['endTime'][sleep_nr])
        )
        
        new_df_filtered_single = df_bpm.loc[condition]
#         display(new_df_filtered_single)
        new_df_filtered_single['sleep_id'] = list(np.repeat(number, len(new_df_filtered_single)))
        new_df_filtered = pd.concat([new_df_filtered, new_df_filtered_single], axis=0)
        number+=1
# Reset the index of the final DataFrame if needed
new_df_filtered.reset_index(drop=True, inplace=True)


In [ ]:
new_df_filtered

In [ ]:
print('All the bpm data: ', len(df_with_all_sleep_data_for_iteration_conf),
     'Only sleep data: ', len(new_df_filtered), 'percentage removed: ', 
      100 - (len(new_df_filtered)*100)/len(df_with_all_sleep_data_for_iteration_conf))

In [ ]:
new_df_filtered.to_pickle(f'{path_to_save}/sleep_segments_all_iterations.pkl')
df_long_sleep.to_pickle(f'{path_to_save}/long_sleep_all_iterations.pkl')
df_night_sleep.to_pickle(f'{path_to_save}/df_night_sleep_all_iterations.pkl')
